In [8]:
import os
from dataclasses import dataclass
from pathlib import Path

In [15]:
%pwd

'd:\\text_summarizer\\notebook'

In [16]:
os.chdir("../")

In [17]:
%pwd

'd:\\text_summarizer'

In [18]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [19]:
from text_summarizer.constants import *
from text_summarizer.utils.common import read_yaml, create_directories

In [24]:
class ConfigurationManager:
    def __init__(self, config_file_path = CONFIG_FILE_PATH, params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        try:
            config = self.config.data_ingestion
            create_directories([config.root_dir])
            data_ingestion_config = DataIngestionConfig(
                root_dir=config.root_dir,
                source_URL=config.source_URL,
                local_data_file=config.local_data_file,
                unzip_dir=config.unzip_dir,
            )
            return data_ingestion_config
        except KeyError as e:
            raise ValueError(f"Missing key in data_ingestion configuration: {e}")


In [21]:
import os
import urllib.request as request
import zipfile
from text_summarizer.logging import logger
from text_summarizer.utils.common import get_size

In [27]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            file_name, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file,
            )
            logger.info(f"{file_name} downloaded with \n {headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_file:
            zip_file.extractall(path=unzip_path)
            logger.info(f"Zip file extracted to {unzip_path}")

In [28]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-12-23 19:40:56,653]: INFO - yaml file: config\config.yaml loaded successfully
[2024-12-23 19:40:56,656]: INFO - yaml file: params.yaml loaded successfully
[2024-12-23 19:40:56,656]: INFO - created directory at: artifacts
[2024-12-23 19:40:56,656]: INFO - created directory at: artifacts/data_ingestion
[2024-12-23 19:40:56,661]: INFO - File already exists of size: ~ 10297 KB
[2024-12-23 19:40:56,727]: INFO - Zip file extracted to artifacts/data_ingestion
